## Autor: Krystian Kasprów (indeks: 226776)

# NN_VoiceRecognition

System rozpozanwania głosu. Głównym założeniem jest rozpoznanie głosu założyciela projektu. 
Sieć będzie mieć zatem dwa wyjścia (jedno dla poprawnego rozpoznania głosu, drugie dla pozostałych dźwięków lub innej osoby). 
Jest to projekt treningowy mający na celu zaznajomienie się ze sztucznymi sieciami neuronowymi.
Projekt realizowany jest przy użyciu narzędzi takich jak:
- język programowania: Python 3
- biblioteka obsługi dźwięku: librosa
- biblioteka do sieci neuronowej: pytorch
- środowisko projektu: jupyter notebook

### Uwagi do projektu: 
- projekt prowadzony jest na github - link do repozytorium: https://github.com/226776/NN_VoiceRecognition
- komentarze w kodzie w celu praktyki napisane są w języku angielskim

### Tutorials (źródła wiedzy):

- https://pytorch.org/tutorials/beginner/nn_tutorial.html (main)

- https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py (main 2)

- https://www.youtube.com/watch?v=LgFNRIFxuUo

- https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

# Proces uczenia Cz. 1
## Klasa VoiceSamples

Pierwszym krokiem jest przygotowanie danych wejściowych (nagrania z telefonu).
Odpowiada za to klasa VoiceSamples.

Metoda __LoadSoundSamples()__ : ładuje ona próbki o rdzeniu nazwy (__core_name__) iterując kolejno od jedynki.
Domyślnie przeszukuje katalog projektu, możliwe jest określenie innej ścieżki poprzez podanie __smaples_path__.

Metoda __ChopToOneSecFragments()__ : dzieli wczytane nagrania na jedno sekundowe 
fragmenty i dodaje je do tablicy wyników jeśli dane fragmenty nie są szumem. 

Metoda __ChopedSignalsToTenosor()__ : liczy transformatę __stft__ i przekształca ją na tensor o wymiarach __256x256__.
Rozdzielczość dobrano eksperymentalnie oraz sprowadzono do najbliższej potęgi dwójki. 

In [ ]:
import librosa as lr
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import logging

from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot  


class VoiceSamples(Dataset):
    
    def __init__(self, core_name, samples_path=None, Automatic=None):
        
        self.Log = logging.getLogger()
        logging.basicConfig(level=logging.INFO)
        
        self.noiseThreshold = 1
        
        self.core_name = core_name
        self.samples_path = samples_path
        
        self.soundSamples = []
        self.sampleRate = []
        self.path = []
        
        self.chopedSamples = []
        self.chopedSr = []
        
        self.tensorMelgrams = []
        
        
        self.info = " VoiceSamples Object successfully created "
        self.Log.info(self.info)
        
        
        if Automatic:
            self.LoadSoundSamples()
            self.ChopToOneSecFragments()
            self.ChopedSignalsToTenosor()
        
    def __len__(self):
        return len(self.tensorMelgrams)
    
    def __getitem__(self, idx):
        if self.tensorMelgrams:
            return self.tensorMelgrams[idx]

    def LoadSoundSamples(self):
    
        n = 1

        while(True):
            try:
                if  self.samples_path:
                    path =  self.samples_path + self.core_name + str(n)
                else:
                    path = self.core_name + str(n)

                soundSample, sampleRate = lr.load(path)

                n += 1
                self.soundSamples.append(soundSample)
                self.sampleRate.append(sampleRate) 
                self.path.append(path)

                self.info = " Sample : " + path + " : successfully added"
                self.Log.info(self.info)

            except FileNotFoundError:
                if self.soundSamples:
                    self.info = "That's the end of database : " + str(n-1) + " : Samples added"
                    self.Log.info(self.info)
                    n = 0
                    
                    return self.soundSamples, self.sampleRate, self.path

                else:
                    self.Log.exception("Files are missing")
                    n = 0

                break

            except Exception as ex:      
                self.Log.exception("Unexpected error")
                break
        
    def getSoundSample(self, idx):
        return self.soundSamples[idx], self.sampleRate[idx]
    
    def getSoundSampleLen(self):
        try:
            if len(self.soundSamples) == len(self.sampleRate):
                return len(self.soundSamples)
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.Log.exception("Unexpected error" + e)
    
    def ChopToOneSecFragments(self):
        
        # TODO: make shure user goes step by step 
        
        try:
            if len(self.soundSamples) == len(self.sampleRate):
                for idx in range(len(self.soundSamples)):
                    
                    soundSample = self.soundSamples[idx]
                    sr = self.sampleRate[idx]
                    
                    frag_max = math.trunc(len(soundSample)/float(sr))
                    step = math.trunc(sr/2);
                    last_sample = len(soundSample)

                    for frag in range(frag_max*2):
                        start = step * frag
                        stop = start + sr
                        if sr<len(soundSample):
                            if self.checkIfNotNoise(soundSample[start:stop]):
                                self.chopedSamples.append(soundSample[start:stop])
                                self.chopedSr.append(sr)
                                self.info = self.path[idx] + " : " + str(frag+1) + " : successfully choped"
                                self.Log.info(self.info)
                            else:
                                self.info = self.path[idx] + " : " + str(frag+1) + " : NOISE!"
                                self.Log.info(self.info)
                        else:
                            self.Log.warning("Something went wrong")
                            
                    if self.checkIfNotNoise(soundSample[last_sample-sr:last_sample]):
                         # incuding samples cuted by math.trunc() 
                        self.chopedSamples.append(soundSample[last_sample-sr:last_sample])
                        self.chopedSr.append(sr)
                        self.info = self.path[idx] +  " : "  + str(frag_max*2+1) + " : successfully choped"
                        self.Log.info(self.info)
                    else:
                        self.info = self.path[idx] + " : "  + str(frag+1) + " : NOISE!"
                        self.Log.info(self.info)
                
                if self.chopedSamples:
                    self.Log.info("Sucessfully choped all loaded signals and eliminated the noise!")
                    return self.chopedSamples, self.chopedSr 
                    
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.e = "Unexpected error : " + str(e)
            self.Log.exception(self.e)
            
    def getChoped(self, idx):
        return self.chopedSamples[idx], self.chopedSr[idx]
        
    def getChopedLen(self):
        try:
            if len(self.chopedSamples) == len(self.chopedSr):
                    return len(self.chopedSamples)
            else:
                self.Log.warning("Lists: sundSamples and sampleRate are not equal!")
                
        except Exception as e:
            self.Log.exception("Unexpected error" + e)
            
        
    def ChopedSignalsToTenosor(self):
        
        # TODO: make shure user goes step by step 
        
        try:
        
            if len(self.chopedSamples) == len(self.chopedSr):
                for idx in range(len(self.chopedSamples)):

                    # hop length adjusted
                    STFT_signal = np.abs(lr.stft(self.chopedSamples[idx], n_fft = 512, hop_length = round(self.chopedSr[idx]/256))) 
                    STFT_signal = lr.power_to_db(STFT_signal**2,ref=np.max)

                    Melgram = STFT_signal[0:256,0:256]
                    TMelgram = torch.tensor(Melgram)
                    self.tensorMelgrams.append(TMelgram)
                    
                    self.info = " " + self.samples_path +  " : ChopedSample " + str(idx) + " : " + " : converted to tensor"
                    self.Log.info(self.info)
                
                if self.tensorMelgrams:
                    self.Log.info("Sucessfully converted all ChopedSamples to Tensors!")
                    return self.tensorMelgrams
                
            else:
                self.Log.warning("Lists: chopedSamples and chopedSr are not equal!")
                
        except Exception as e:
            self.e = "Unexpected error : " + str(e)
            self.Log.exception(self.e)
                
    
    
    def checkIfNotNoise(self, chopedSample):
    
        chopedSamplePow2 = []

        for n in range(len(chopedSample)):
            chopedSamplePow2.append(chopedSample[n]**2)
        sk = sum(chopedSamplePow2)
        if sk > self.noiseThreshold:
            return True 
        else:
            return False

## Klasa VoiceSamplesInput()

Klasa wykorzystuje wyżej opisaną klasę __VoiceSamples()__ do załadowania próbek dwóch ludzi oraz przypisania im odpowiedniego spodziewanego wyniku (__target__).

Sieć neuronowa nie powinna uczyć się zbyt uporządkowanych danych, dlatego też dane dwóch osób są podawane naprzemiennnie. 
Klasa dla wejściowego indeksu parzystego zwraca próbki osoby pierwszej, dla nieparzystego natomiast osoby drugiej.

In [ ]:


class VoiceSamplesInput():
    def __init__(self):
        
        # load voice samples of Krystian
        self.vsKrystian = VoiceSamples("vsKrystian", samples_path="database/Krystian/" , Automatic=True)
        # load voice samples of Nicia
        self.vsNicia = VoiceSamples("vsNicia", samples_path="database/Nicia/" , Automatic=True)
        
        # expected output - target of Krystian
        self.targetKrystian = torch.tensor([[float(1),float(0)]])
        # expected output - target of Nicia
        self.targetNicia = torch.tensor([[float(0),float(1)]])
        
        
    def __getitem__(self, idx):
        if idx % 2 == 0:
            return self.vsKrystian[int(idx/2)] ,  self.targetKrystian
        else:
            return self.vsNicia[int((idx+1)/2)] , self.targetNicia
    
    def __len__(self):
        if len(self.vsKrystian) <= len(self.vsNicia):
            return len(self.vsKrystian) * 2
        else:
            return len(self.vsNicia) * 2
            




## VoiceRecogModel

Model sieci neuronowej wykorzystywany do reazlizacji klasyfikacji rozpoznania właściciela nagranego głosu. 

Model składa się z pięciu warstw ukrytych (nie licząc tzw. pooling'u) :
- dwie warstwy splotowe (convolutional): __conv1d__, __conv2d__
- trzy warstwy w pełni połączone (fully connected): __fc1__, __fc2__, __fc3__

Dane na wyjściu warstw splotowych są redukowane dwukrotnie przy zachowaniu, 
czy też nawet wzmocnieniu najważniejszych cehch przy użyciu funkcji __F.max_pool2d()__. 

Funkcją aktywacji jest funkcja __F.relu()__.

Model przeliczony jest dla jednego tensora wejściowego o wymiarach __256x256__. 
W praktyce oznacza to zbadanie/wyliczenie wymiarów pojedynczego wyjścia warstwy __conv2d (62x62)__.
Wejście kolejenej warstwy  __fc1__ jest więc iloczynem wymiarów pojedynczego wyjścia oraz ilością kanałów (__20__).

Sieć z racji realizowanego zadania ma jedynie 2 wyjścia __fc3__. 

Metoda __forward(self, x)__ relizuje przeliczenie pojedynczego obiektu wejsciowego na oczekiwane wyjście. 

Uwaga! Wszystkie parametry sieci zostały dobrane eksperymentalnie w oparciu o przykłady oraz zasadę, że ilość neuronów w warstwie powinna być pomiędzy ilościami neuronów w warstwach sąsiednich.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VoiceRecogModel(nn.Module):

    def __init__(self):
        super(VoiceRecogModel, self).__init__()
        # 1 input image channel, 10 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 10, 3)
        self.conv2 = nn.Conv2d(10, 20, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(20*62*62, 2000)  # 20: Conv2d output channel number, 62x62: Conv2d one channel image
        self.fc2 = nn.Linear(2000, 300)
        self.fc3 = nn.Linear(300, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

W tym miejscu tworzona jest baza do uczenia sieci.

In [ ]:
import torch.optim as optim

# create dataset : loading and processing samples to tensors
vsInput = VoiceSamplesInput()


W tym miejscu tworzony jest model sieci, funkcja strat oraz funkcja realizująca wsteczną propagację (__optimizer__).

Krytycznym jest odpowiednie dobranie prametru __lr__ określającego wpływ każdej iteracji na korekcję wag.
Im większy __lr__, tym większa korekta. 
Zbyt duży __lr__ powodują, iż wyjście będzie zbiegać do nieskonczoności lub sieć będzie rozpoznawać jedynie kilka - kilkanaście ostatnich obiektów wejściowych.
Zbyt mały __lr__, spowoduje, że sieć do nauki będzie wymagała ogromną ilość danych.

In [ ]:
# create net from VoiceRecogModel
net = VoiceRecogModel()

# loss function (using function implemented in pytorch)
criterion = nn.MSELoss()

# create your optimizer (basic optimizer)
# setting learning rate 
optimizer = optim.SGD(net.parameters(), lr=0.001)

Preces uczenia (realizowany poniżej) polega na :
    - wyliczeniu wyjścia dla pobranej z bazy próbki (__vs, target = vsInput[k]__)
    - wyliczeniu straty, czyli różnicy pomiędzy wyjściem a oczekiwaniami (__loss = criterion(output, target)__)
    - wyliczeniu tzw. biasu, czyli wartości o jakie skorygowane mają zostać wagi (__loss.backward()__)
    - zaktualizowaniu wag
    
Proces ten powtarzany jest dla wszystkich elementów bazy. Przy użyciu tej samej bazy można uczyć klika razy (np. __epoch = 2__).


In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
trainLog = logging.getLogger()

# Training Loop

# How many times learn on the same dataset
epoch = 1
for i in range(epoch):   
    for k in range(len(vsInput)):  
        try:
            vs, target = vsInput[k]

            optimizer.zero_grad()   # zero the gradient buffers
            input = vs.view(-1,1,256,256)
            output = net(input)

            loss = criterion(output, target)

            loss.backward()
            optimizer.step()    # Does the update


            info = "Training " + str(k)+"/"+str(len(vsInput))+" done"

            print("\n")
            print(target)
            print(output)
            print(loss)
            print(info)
        except Exception as e:
            print(e)
            pass

Zapisanie modelu sieci na dysku lokalnym. 